# Modèle de NLP avec NLTK

## 1. Librairies

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Dataset metrics

In [10]:
# nom de fichier et chemin relatif
filename = '/raw/metrics.csv'
path = '../data'
# création d'un dataframe à partir du csv de données
df = pd.read_csv(path+filename, index_col=0)
df.head(2)

,status,created_at,cyan_capacity,cyan_remaining,magenta_capacity,magenta_remaining,yellow_capacity,yellow_remaining,black_capacity,black_remaining,machineId,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
id,,,,,,,,,,,,,,,,
5561325,WARNING,2022-06-07 12:23:52.164000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,"[{""name"": ""JAN"", ""level"": ""Operator""}]",22050.818386,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
5513803,IDLE,2022-06-03 08:44:57.996000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",22402.256226,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164430 entries, 5561325 to 10909319
Data columns (total 16 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   status                     1164430 non-null  object 
 1   created_at                 1164430 non-null  object 
 2   cyan_capacity              0 non-null        float64
 3   cyan_remaining             0 non-null        float64
 4   magenta_capacity           0 non-null        float64
 5   magenta_remaining          0 non-null        float64
 6   yellow_capacity            0 non-null        float64
 7   yellow_remaining           0 non-null        float64
 8   black_capacity             0 non-null        float64
 9   black_remaining            0 non-null        float64
 10  machineId                  1164430 non-null  int64  
 11  connected_operators        1164430 non-null  object 
 12  varnishLevelsTargetvolume  1164430 non-null  float64
 13  varni

In [12]:
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df = df.dropna(axis=1)

In [13]:
df.head(2)

,status,created_at,machineId,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
0,WARNING,2022-06-07 12:23:52.164000,14,"[{""name"": ""JAN"", ""level"": ""Operator""}]",22050.818386,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
1,IDLE,2022-06-03 08:44:57.996000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",22402.256226,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


## 3. Colonne Modules

### a) Tokenisation des mots

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\odazo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
str_ = df.modules.loc[0]
words_in_quote = word_tokenize(str_)
words = [word.lower() for word in words_in_quote if word.isalpha() or word.isnumeric()]
print(words)
len(words)

['sn', 'name', 'print', 'engine', '1', 'type', 'varnish', 'printer', 'counters', 'name', 'varnish', 'counter', 'value', '1965087', 'generation', 'sn', 'name', 'ifoil', 'l', 'type', 'ifoil', 'counters', 'name', 'total', 'pages', 'counter', 'value', '63822', 'name', 'foiled', 'pages', 'counter', 'value', '36211', 'generation', '2']


36

### b) Vectorisation des tokens

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialiser le vecteur TF-IDF
tf_idf_vector = TfidfVectorizer()
# Vectoriser les mots
word_vector = tf_idf_vector.fit_transform(words)
# Afficher le vecteur résultant
print(f'le token "{words[1]}" = un vecteur de taille {word_vector[1]}')
word_vector.shape

le token "sn" = un vecteur de taille   (0, 13)	1.0


(36, 18)

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
from numpy import count_nonzero

# representing in CSR form
X_sparse = csr_matrix(word_vector)
# specify the no of output features
tsvd = TruncatedSVD(n_components=10)
# apply the truncatedSVD function
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)
# shape of the reduced matrix
print(X_sparse_tsvd.shape)


(36, 10)


In [30]:
def vectorize_serie(serie):
    tfid_vectorize = TfidfVectorizer()
    words = [word.lower() for word in word_tokenize(serie) if word.isalpha() or word.isnumeric()]
    word_vector = tfid_vectorize.fit_transform(words)
    return word_vector.resize(1,1)

In [31]:
# plus de 20min
module_df = df.modules.apply(lambda x : vectorize_serie(x))


KeyboardInterrupt: 

In [37]:
#module_df = df.modules.apply(lambda x : [word.lower() for word in word_tokenize(x) if word.isalpha() or word.isnumeric()])

In [53]:
module_df.loc[0].get_shape()

(36, 18)

In [103]:
def sparce_vector(serie):
    X_sparse = csr_matrix(serie)
    #print(X_sparse)

    # specify the no of output features
    tsvd = TruncatedSVD(n_components=10)

    # apply the truncatedSVD function
    X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)
    #print(X_sparse_tsvd)

    # shape of the reduced matrix
    #print(X_sparse_tsvd.shape)

    return X_sparse_tsvd

In [104]:
sparce_module_df = module_df.apply(lambda x : sparce_vector(x))

In [108]:
sparce_module_df.loc[0]

array([[ 3.20692026e-16, -1.39862080e-16, -9.28541275e-17,
        -4.21614222e-01, -7.06408444e-02,  5.90430653e-01,
        -1.85051299e-02, -4.14600244e-01,  3.69906998e-01,
         3.99469615e-01],
       [ 1.00000000e+00, -6.19700807e-15,  2.30476165e-16,
        -8.32667268e-17,  2.69360552e-15, -1.66533454e-16,
         1.11022302e-16, -1.38777878e-16, -2.22044605e-16,
        -2.22044605e-16],
       [ 5.88412856e-17, -1.91860416e-15, -3.27968334e-17,
        -3.56794294e-16,  2.58126853e-15,  2.62021141e-15,
        -6.54369824e-15, -3.27372561e-18,  2.44217171e-16,
        -5.21318035e-16],
       [-1.49294711e-16, -5.75928194e-16, -1.09985792e-16,
        -6.90732718e-17,  1.22124533e-15, -1.00993291e-15,
        -5.88510675e-15, -2.28870348e-16,  5.60651389e-17,
         7.03992366e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.